# **Exploração e análise de dados de crédito com SQL**


## Os dados: 

Os dados representam informações de clientes de um banco e contam com as seguintes colunas: 

- id = identificador cliente  
- idade = idade do cliente
- sexo = sexo do cliente (F ou M)
- dependentes = número de dependentes do cliente
- escolaridade = nível de escolaridade do clientes
- estado_civil = estado civil do cliente 
- salario_anual = faixa salarial do cliente
- tipo_cartao = tipo de cartao do cliente
- meses_relacionamento = meses de relacionamento do cliente com o banco 
- qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
- iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
- meses_inativo_12m = quantidade de meses que o cliente ficou inativo
- limite_credito = limite de credito do cliente
- valor_transacoes_12m = valor das transações dos ultimos 12 meses
- qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses

A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** 


## **Exploração de dados:**

A primeira fase da análise é entender o que tem na nossa matéria prima. Vamos a exploração de dados:

**Qual a quantidade de informações temos na nossa base de dados?**

Query: SELECT count(*) FROM credito
> Reposta: 10127 linhas

**Como são os dados?** 

Query: SELECT * FROM credito LIMIT 10;
![Dez primeiras linhas do dataset](https://github.com/davicarv/sql_ebac/blob/main/LIMIT10.png?raw=true)

> É possível reparar que existem algumas informações nulas na tabela (valor na), vamos olhar mais de perto para os valores de cada coluna! 

**Existem dados duplicados no dataset?**

Query: SELECT COUNT(DISTINCT id) AS valores_id FROM credito_parquet:

> Resposta: 10127

Então nenhum cliente foi computado duas vezes na fonte de dados, temos 10127 clientes distintos na tabela.

**Quais são os tipos de cada dado?**

**Query:** DESCRIBE credito

![Descrição dos tipos de dados](https://github.com/davicarv/sql_ebac/blob/main/DESCRIBE.png?raw=true)

Agora que ja entendemos quais são os tipos de dados, vamos olhar mais atentamente para as varíaveis que não são numéricas.

**Quais são os tipos de escolaridade disponíveis no dataset?**

**Query:** SELECT DISTINCT escolaridade FROM credito

![Tipos de escolaridade](https://github.com/davicarv/sql_ebac/blob/main/ESCOLARIDADE.png?raw=true)

> Os dados contam com vários níveis de escolaridade e é possível perceber que temos valores nulos (na) no dataset, trataremos isso mais tarde!

Será que temos mais valores nulos?

**Quais são os tipos de estado_civil disponíveis no dataset?**

**Query:** SELECT DISTINCT estado_civil FROM credito

![Tipos de estado civil](https://github.com/davicarv/sql_ebac/blob/main/ESTADO_CIVIL.png?raw=true)

> Novamente encontramos valores nulos nos dados de estado civil!

**Quais são os tipos de salario_anual disponíveis no dataset?**

**Query:** SELECT DISTINCT salario_anual FROM credito

![Tipos de salario anual](https://github.com/davicarv/sql_ebac/blob/main/SALARIO_ANUAL.png?raw=true)

> Os salários nesse dataset não estão expostas com o valor exato que o cliente ganha. O que está informado é a faixa salarial de cada um. Também contem dados nulos. 


**Quais são os tipos de cartão disponíveis no dataset?**

**Query:** SELECT DISTINCT tipo_cartao  FROM credito

![Tipos de cartão](https://github.com/davicarv/sql_ebac/blob/main/TIPO_CARTAO.png?raw=true)

> Aqui vemos que não há necessidade de tratar valores nulos.


## **Análise de dados**

Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados. Vamos fazer perguntas:
/
**Nesse banco de dados, quantos clientes temos de cada faixa salarial?**

**Query:** SELECT COUNT(*) AS quantidade, salario_anual FROM credito GROUP BY salario_anual

![Quantidade para cada faixa salarial](https://github.com/davicarv/sql_ebac/blob/main/FAIXA_SALARIAL.png?raw=true)

> A maioria dos clientes dessa base de dados possui um renda menor que 40K e existem 1112 clientes que não informaram ou não consta a faixa salarial. É possível perceber que, quanto maior a faixa salarial analisada, menos clientes. 
> De certa forma, pode ser interessante para a empresa focar nesse público de mais baixa renda. 

**Nesse banco de dados, quantos clientes são homens e quantos são mulheres?**

**Query:** SELECT COUNT(*) AS quantidade, sexo FROM credito GROUP BY sexo


![Quantidade para cada sexo](https://github.com/davicarv/sql_ebac/blob/main/QUANT_SEXO.png?raw=true)

![Quantidade para cada sexo - gráfico](https://github.com/davicarv/sql_ebac/blob/main/GRAFICO_SEXO.png?raw=true)

> A maioria dos clientes desse banco é mulher! Do csv extraído dos dados é possível gerar o gráfico em pizza com para melhor a proporção de masculino/feminino

**Queremos focar o nosso marketing de maneira adequada para nossos clientes, qual será a idade deles?**

**Query:** SELECT AVG(idade) AS media_idade, MIN(idade) AS min_idade, MAX(idade) AS max_idade, sexo FROM credito GROUP BY sexo
![Média de idades por sexo](https://github.com/davicarv/sql_ebac/blob/main/IDADE_SEXO.png?raw=true)

> Por meio dessa análise não foi possível extrair nenhuma informação relevante.  A menor idade dos dois sexos é a mesma e a média é muito similar. A unica  diferença é a idade máxima mas fato quase irrelevante por a diferença não é tão gritante.


**Qual a distribuição dos valores de transação dos clientes?**

**Query:** SELECT MIN(valor_transacoes_12m) AS min_valor,approx_percentile(valor_transacoes_12m, 0.25) AS primeiro_quartil, approx_percentile(valor_transacoes_12m, 0.50) AS mediana,approx_percentile(valor_transacoes_12m, 0.75) AS terceiro_quartil, MAX(valor_transacoes_12m) AS max_valor, FROM credito;

![Valor transacoes](https://github.com/davicarv/sql_ebac/blob/main/TRANSAÇOES.png?raw=true)

> Nesse banco de dados temos que 75% das somas de transações em 12 meses variam de 510.16 a 5776.58. Já os 25% maiores valores variam de 11144.88 a 34516.99. 

**Qual a distribuição dos valores de limites dos clientes?**

**Query:** SELECT MIN(limite_credito) AS min_valor,approx_percentile(limite_credito, 0.25) AS primeiro_quartil, approx_percentile(limite_credito, 0.50) AS mediana,approx_percentile(limite_credito, 0.75) AS terceiro_quartil, MAX(limite_credito) AS max_valor, FROM credito;

![Valor transacoes](https://github.com/davicarv/sql_ebac/blob/main/LIMITES.png?raw=true)

> Nesse banco de dados temos que 75% dos limites de crédito variam de 1438 a 11144.88. Já os 25% maiores valores variam de 11144.88 a 34516.99. 

**Quais as características dos clientes que possuem os maiores creditos?**

**Query:** 
SELECT max(limite_credito) AS limite_credito, escolaridade, tipo_cartao, sexo 
FROM credito
WHERE escolaridade != 'na' AND tipo_cartao != 'na' 
GROUP by  escolaridade, tipo_cartao, sexo 
ORDER by limite_credito desc
LIMIT 10

![Valor limite](https://github.com/davicarv/sql_ebac/blob/main/LIMITE_DESC.png?raw=true)

> Não parece haver um impacto da escolaridade no limite. O limite mais alto é oferecido para um homem com ensino_médio e dois sem educação formal. O cartão também parece não estar relacionado com a escolaridade nem com o limite. Dentre os maiores limites, encontramos clientes com cartão: gold, silver, platinum e blue.
> Apenas uma mulher está entre os 10 clientes com maior limite, o que pode significar uma relação entre limite e sexo.

**Quais as características dos clientes que possuem os menores creditos?**

**Query:** 
SELECT min(limite_credito) AS limite_credito, escolaridade, tipo_cartao, sexo 
FROM credito
WHERE escolaridade != 'na' AND tipo_cartao != 'na' 
GROUP by  escolaridade, tipo_cartao, sexo 
ORDER by limite_credito asc
LIMIT 10

![Valor limite](https://github.com/davicarv/sql_ebac/blob/main/LIMITE_ASC.png?raw=true)

> Dessa vez conseguimos perceber que só há clientes com cartão blue dentre os menores limites. Também foi possível perceber que os 10 menores limites são igualmente distribuídos entre homens e mulheres.

**Qual a diferença de limites dos clientes por sexo?**

**Query:** SELECT sexo, MIN(limite_credito) AS min_valor,approx_percentile(limite_credito, 0.25) AS primeiro_quartil, approx_percentile(limite_credito, 0.50) AS mediana,approx_percentile(limite_credito, 0.75) AS terceiro_quartil, MAX(limite_credito) AS max_valor FROM credito GROUP BY sexo

![Limite por sexo](https://github.com/davicarv/sql_ebac/blob/main/LIMITE_SEXO.png?raw=true)

> Conseguimos perceber que os homens tem uma tendência de conseguirem limites bem mais altos que mulheres 

**Será que as mulheres gastam mais?**

**Query:** 
SELECT MAX(valor_transacoes_12m) AS maior_valor_gasto, AVG(valor_transacoes_12m) AS  media_valor_gasto, MIN(valor_transacoes_12m) AS min_valor_gasto, sexo FROM credito GROUP BY sexo 

![Valor transacoes/sexo](https://github.com/davicarv/sql_ebac/blob/main/GASTO_SEXO.png?raw=true)

> Apesar da diferença nos limites, os gastos de homens e mulheres são similares!

Por fim, 

**O salário impacta no limite?**

**Query:** SELECT AVG(valor_transacoes_12m) AS media_valor_transacoes, AVG(limite_credito) AS media_limite,  sexo,   salario_anual FROM credito 
WHERE salario_anual != 'na'
GROUP BY sexo, salario_anual
ORDER BY avg(valor_transacoes_12m) DESC

![Valor salario_anualLimite](https://github.com/davicarv/sql_ebac/blob/main/LIMITE_SALARIO.png?raw=true)

>SIM! As pessoas que tem menor faixa salarial também apresentam menor limite de credito! Além disso, não há mulheres com salários declarados maior que 60k, o qye pode explicar o fato de homens possuirem limites bem mais altos.  









# Conclusão

Essas foram **algumas** análises extraídas do dataset de crédito.  

Alguns insights interessantes:

- a maior parte dos clientes possui renda até 40K 
- a maior parte dos clientes é feminino!
- a escolaridade não parece influenciar no limite nem no tipo do cartão
- os clientes com maiores limites são em sua maioria homens 
- os clientes com menores limites não são relacionados ao sexo do cliente
- dentre os menores limites predomina o cartão tipo blue
- por um lado, homens possuem limites maiores que mulheres, por outro lado, também possuem salários maiores 
- a faixa salarial impacta diretamente no limite de crédito
- nao existem clientes com salário anual acima de 60K do sexo feminino


